In [2]:
!pip install zeugma
!pip install deslib

In [3]:
from zeugma.embeddings import EmbeddingTransformer

Using TensorFlow backend.


In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score

In [5]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from preprocessing import *
from function import *

In [7]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from deslib.util.aggregation import *

In [8]:
w2v = EmbeddingTransformer('word2vec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [9]:
glove = EmbeddingTransformer('glove')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [10]:
fasttext = EmbeddingTransformer('fasttext')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [11]:
uri_train = 'https://raw.githubusercontent.com/wvs2/data-hate/master/union/train.csv'
uri_val   = 'https://raw.githubusercontent.com/wvs2/data-hate/master/union/val.csv'
uri_teste = 'https://raw.githubusercontent.com/wvs2/data-hate/master/union/test.csv'

df_train = pd.read_table(uri_train, sep=',')
df_val   = pd.read_table(uri_val, sep=',')
df_test = pd.read_table(uri_teste, sep=',')

In [12]:
train = df_train['text'].fillna(' ').apply(pre_processing)
val = df_val['text'].fillna(' ').apply(pre_processing)
test, class_test = df_test['text'].fillna(' ').apply(pre_processing), df_test['class']

In [13]:
cv = CountVectorizer(analyzer='word', lowercase=True, stop_words='english')
cv.fit_transform(train.values.astype('U'))
     
tfidf =  TfidfVectorizer(analyzer='word', lowercase=True, use_idf=True, stop_words='english')
tfidf.fit_transform(train.values.astype('U'))

<20187x18542 sparse matrix of type '<class 'numpy.float64'>'
	with 138131 stored elements in Compressed Sparse Row format>

In [14]:
svm = {
    'CV': {
        'CLF': SVC(random_state=42, verbose=100, kernel='linear', gamma=0.1, probability=True),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': SVC(random_state=42, verbose=100, kernel='linear', gamma=0.1, probability=True),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': SVC(random_state=42, verbose=100, kernel='rbf', gamma=1, probability=True),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': SVC(random_state=42, verbose=100, kernel='rbf', gamma=0.5, probability=True),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': SVC(random_state=42, verbose=100, kernel='rbf', gamma=1, probability=True),
        'EXT': fasttext,
    }
}

In [15]:
svm_clfs = {

}
for ext, clf in svm.items():
    svm_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [16]:
df_pred_train = pd.DataFrame(df_train['class'])
df_pred_val = pd.DataFrame(df_val['class'])
df_pred_test = pd.DataFrame(df_test['class'])

df_prob_val = pd.DataFrame(df_val['class'])
df_prob_train = pd.DataFrame(df_train['class'])
df_prob_test = pd.DataFrame(df_test['class'])
for ext, clf in svm_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["SVM-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["SVM-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["SVM-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "SVM-{}-{}".format(ext,clf.classes_[0]), 
    "SVM-{}-{}".format(ext,clf.classes_[1]), 
    "SVM-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [17]:
lr = {
    'CV': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': LogisticRegression(random_state=42, verbose=100, multi_class='auto', solver='liblinear', penalty='l1'),
        'EXT': fasttext,
    }
}

In [18]:
lr_clfs = {

}
for ext, clf in lr.items():
    lr_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

[LibLinear][LibLinear][LibLinear][LibLinear][LibLinear]

In [19]:
for ext, clf in lr_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["LR-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["LR-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["LR-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "LR-{}-{}".format(ext,clf.classes_[0]), 
    "LR-{}-{}".format(ext,clf.classes_[1]), 
    "LR-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [20]:
rf = {
    'CV': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': RandomForestClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': fasttext,
    }
}

In [21]:
rf_clfs = {

}
for ext, clf in rf.items():
    rf_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 50
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s
building tree 2 of 50
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.6s remaining:    0.0s
building tree 3 of 50
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.8s remaining:    0.0s
building tree 4 of 50
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.1s remaining:    0.0s
building tree 5 of 50
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    1.4s remaining:    0.0s
building tree 6 of 50
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    1.7s remaining:    0.0s
building tree 7 of 50
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    1.9s remaining:    0.0s
building tree 8 of 50
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    2.2s remaining:    0.0s
building tree 9 of 50
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.5s remaining:    0.0s
b

In [22]:
for ext, clf in rf_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["RF-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["RF-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["RF-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "RF-{}-{}".format(ext,clf.classes_[0]), 
    "RF-{}-{}".format(ext,clf.classes_[1]), 
    "RF-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

In [23]:
nb = {
    'CV': {
        'CLF': MultinomialNB(alpha=0.5, fit_prior=False),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': MultinomialNB(alpha=0.5, fit_prior=False),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': BernoulliNB(alpha=0.5, fit_prior=True),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': BernoulliNB(alpha=0.1, fit_prior=True),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': BernoulliNB(alpha=0.1, fit_prior=True),
        'EXT': fasttext,
    }
}

In [24]:
nb_clfs = {

}
for ext, clf in nb.items():
    nb_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

In [25]:
for ext, clf in nb_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["NB-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["NB-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["NB-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "NB-{}-{}".format(ext,clf.classes_[0]), 
    "NB-{}-{}".format(ext,clf.classes_[1]), 
    "NB-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [26]:
mlp = {
    'CV': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='lbfgs'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': MLPClassifier(random_state=42, batch_size=20, max_iter=20, verbose=100, activation='relu', solver='adam'),
        'EXT': fasttext,
    }
}

In [27]:
mlp_clfs = {

}
for ext, clf in mlp.items():
    mlp_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Iteration 1, loss = 0.46594565
Iteration 2, loss = 0.18186529
Iteration 3, loss = 0.10755989
Iteration 4, loss = 0.07418390
Iteration 5, loss = 0.05703002
Iteration 6, loss = 0.04677669
Iteration 7, loss = 0.04013424
Iteration 8, loss = 0.03615691
Iteration 9, loss = 0.03319095
Iteration 10, loss = 0.03137551
Iteration 11, loss = 0.02981392
Iteration 12, loss = 0.02857431
Iteration 13, loss = 0.02693112
Iteration 14, loss = 0.02584594
Iteration 15, loss = 0.02495694
Iteration 16, loss = 0.02432573
Iteration 17, loss = 0.02342970
Iteration 18, loss = 0.02249436
Iteration 19, loss = 0.02319899
Iteration 20, loss = 0.02290472


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.56234007
Iteration 2, loss = 0.48357596
Iteration 3, loss = 0.46283934
Iteration 4, loss = 0.44961013
Iteration 5, loss = 0.43876335
Iteration 6, loss = 0.42743227
Iteration 7, loss = 0.41636061
Iteration 8, loss = 0.40662484
Iteration 9, loss = 0.39565882
Iteration 10, loss = 0.38657991
Iteration 11, loss = 0.37763563
Iteration 12, loss = 0.36741490
Iteration 13, loss = 0.35737291
Iteration 14, loss = 0.34746482
Iteration 15, loss = 0.33750485
Iteration 16, loss = 0.32945826
Iteration 17, loss = 0.31877518
Iteration 18, loss = 0.31045807
Iteration 19, loss = 0.30275097
Iteration 20, loss = 0.29140432


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.63485303
Iteration 2, loss = 0.59026133
Iteration 3, loss = 0.57920443
Iteration 4, loss = 0.57440423
Iteration 5, loss = 0.56759280
Iteration 6, loss = 0.56377882
Iteration 7, loss = 0.55918638
Iteration 8, loss = 0.55654215
Iteration 9, loss = 0.55351645
Iteration 10, loss = 0.55133111
Iteration 11, loss = 0.54863057
Iteration 12, loss = 0.54590483
Iteration 13, loss = 0.54487559
Iteration 14, loss = 0.54145149
Iteration 15, loss = 0.54184517
Iteration 16, loss = 0.53974832
Iteration 17, loss = 0.53696260
Iteration 18, loss = 0.53670900
Iteration 19, loss = 0.53437168
Iteration 20, loss = 0.53341451


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Iteration 1, loss = 0.60304698
Iteration 2, loss = 0.48236804
Iteration 3, loss = 0.45767942
Iteration 4, loss = 0.44535514
Iteration 5, loss = 0.43627923
Iteration 6, loss = 0.42929631
Iteration 7, loss = 0.42249412
Iteration 8, loss = 0.41756293
Iteration 9, loss = 0.41071985
Iteration 10, loss = 0.40726750
Iteration 11, loss = 0.40244520
Iteration 12, loss = 0.39667310
Iteration 13, loss = 0.39130827
Iteration 14, loss = 0.38660231
Iteration 15, loss = 0.38259697
Iteration 16, loss = 0.37738778
Iteration 17, loss = 0.37270980
Iteration 18, loss = 0.36839515
Iteration 19, loss = 0.36338411
Iteration 20, loss = 0.35791440


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (20) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [28]:
for ext, clf in mlp_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["MLP-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["MLP-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["MLP-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "MLP-{}-{}".format(ext,clf.classes_[0]), 
    "MLP-{}-{}".format(ext,clf.classes_[1]), 
    "MLP-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [29]:
extra = {
    'CV': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=20),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': ExtraTreesClassifier(random_state=42, verbose=100, n_estimators=50),
        'EXT': fasttext,
    }
}

In [30]:
extra_clfs = {

}
for ext, clf in extra.items():
    extra_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
building tree 1 of 50
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.4s remaining:    0.0s
building tree 2 of 50
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.0s remaining:    0.0s
building tree 3 of 50
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.4s remaining:    0.0s
building tree 4 of 50
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    1.9s remaining:    0.0s
building tree 5 of 50
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    2.4s remaining:    0.0s
building tree 6 of 50
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    2.8s remaining:    0.0s
building tree 7 of 50
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    3.2s remaining:    0.0s
building tree 8 of 50
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    3.7s remaining:    0.0s
building tree 9 of 50
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    4.1s remaining:    0.0s
b

In [31]:
for ext, clf in extra_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["EXTRA-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["EXTRA-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["EXTRA-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "EXTRA-{}-{}".format(ext,clf.classes_[0]), 
    "EXTRA-{}-{}".format(ext,clf.classes_[1]), 
    "EXTRA-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   6 out of   6 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   7 out of   7 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   8 out of   8 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  11 out of  11 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  12 out of  

In [32]:
knn = {
    'CV': {
        'CLF': KNeighborsClassifier(n_neighbors=3, algorithm='auto'),
        'EXT': cv,
    },
    'TFIDF': {
        'CLF': KNeighborsClassifier(n_neighbors=3, algorithm='auto'),
        'EXT': tfidf,
    },
    'W2V': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': w2v,
    },
    'GLOVE': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='ball_tree'),
        'EXT':  glove,
    },
    'FAST': {
        'CLF': KNeighborsClassifier(n_neighbors=5, algorithm='auto'),
        'EXT': fasttext,
    }
}

In [33]:
knn_clfs = { }
for ext, clf in knn.items():
    knn_clfs[ext] = get_classifier(clf['CLF'], train, df_train['class'], clf['EXT'])

In [34]:
for ext, clf in knn_clfs.items():
  # Predict
  df_train_ = pd.DataFrame(clf.predict(train), columns=["KNN-{}".format(ext)])
  df_val_ = pd.DataFrame(clf.predict(val), columns=["KNN-{}".format(ext)])
  df_test_ = pd.DataFrame(clf.predict(test), columns=["KNN-{}".format(ext)])
  
  df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
  df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
  df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

  # # Probabilidades
  cols = [
    "KNN-{}-{}".format(ext,clf.classes_[0]), 
    "KNN-{}-{}".format(ext,clf.classes_[1]), 
    "KNN-{}-{}".format(ext,clf.classes_[2])
  ]
  df_train_ = pd.DataFrame(clf.predict_proba(train), columns=cols)
  df_val_ = pd.DataFrame(clf.predict_proba(val), columns=cols)
  df_test_ = pd.DataFrame(clf.predict_proba(test), columns=cols)
  df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
  df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
  df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


# CNN

In [35]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH=300

y_train = to_categorical(df_train['class'])
y_val = to_categorical(df_val['class'])

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(train)

seq_train = tokenizer.texts_to_sequences(train)
seq_val = tokenizer.texts_to_sequences(val)
seq_test = tokenizer.texts_to_sequences(test)

data_train = pad_sequences(seq_train, maxlen=MAX_SEQUENCE_LENGTH)
data_val = pad_sequences(seq_val, maxlen=MAX_SEQUENCE_LENGTH)
data_test = pad_sequences(seq_test, maxlen=MAX_SEQUENCE_LENGTH)

In [36]:
cnn_cv = get_CNN(cv, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid')
cnn_tfidf = get_CNN(tfidf, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='softmax')
cnn_w2v = get_CNN(w2v, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True)
cnn_glove = get_CNN(glove, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=25, activation='sigmoid', word_embedding=True)
cnn_fast = get_CNN(fasttext, tokenizer, MAX_NB_WORDS, EMBEDDING_DIM=300, activation='sigmoid', word_embedding=True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 300, 300)          5816700   
_________________________________________________________________
dropout (Dropout)            (None, 300, 300)          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 298, 64)           57664     
_________________________________________________________________
global_max_pooling1d (Global (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 256)               16640     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
activation (Activation)      (None, 256)               0

In [37]:
cnn_cv.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=20)

Epoch 1/20
1010/1010 [==============================] - 64s 64ms/step - loss: 0.6809 - accuracy: 0.6921 - val_loss: 0.5502 - val_accuracy: 0.7510
Epoch 2/20
1010/1010 [==============================] - 64s 63ms/step - loss: 0.5433 - accuracy: 0.7566 - val_loss: 0.5108 - val_accuracy: 0.7713
Epoch 3/20
1010/1010 [==============================] - 64s 63ms/step - loss: 0.5069 - accuracy: 0.7684 - val_loss: 0.4761 - val_accuracy: 0.7715
Epoch 4/20
1010/1010 [==============================] - 63s 63ms/step - loss: 0.4545 - accuracy: 0.7934 - val_loss: 0.4181 - val_accuracy: 0.8093
Epoch 5/20
1010/1010 [==============================] - 63s 63ms/step - loss: 0.4178 - accuracy: 0.8199 - val_loss: 0.4012 - val_accuracy: 0.8387
Epoch 6/20
1010/1010 [==============================] - 63s 63ms/step - loss: 0.3994 - accuracy: 0.8341 - val_loss: 0.3954 - val_accuracy: 0.8433
Epoch 7/20
1010/1010 [==============================] - 64s 63ms/step - loss: 0.3847 - accuracy: 0.8432 - val_loss: 0.3817 -

In [38]:
cnn_tfidf.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_w2v.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_glove.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)
cnn_fast.fit(data_train, y_train, validation_data=(data_val, y_val), epochs=20, batch_size=200)

Epoch 1/20
101/101 [==============================] - 56s 556ms/step - loss: 0.8778 - accuracy: 0.6319 - val_loss: 0.7647 - val_accuracy: 0.6390
Epoch 2/20
101/101 [==============================] - 56s 555ms/step - loss: 0.6623 - accuracy: 0.6825 - val_loss: 0.5794 - val_accuracy: 0.7478
Epoch 3/20
101/101 [==============================] - 56s 551ms/step - loss: 0.5697 - accuracy: 0.7494 - val_loss: 0.5417 - val_accuracy: 0.7558
Epoch 4/20
101/101 [==============================] - 56s 553ms/step - loss: 0.5419 - accuracy: 0.7567 - val_loss: 0.5181 - val_accuracy: 0.7687
Epoch 5/20
101/101 [==============================] - 56s 552ms/step - loss: 0.5228 - accuracy: 0.7626 - val_loss: 0.4975 - val_accuracy: 0.7689
Epoch 6/20
101/101 [==============================] - 56s 553ms/step - loss: 0.4992 - accuracy: 0.7681 - val_loss: 0.4742 - val_accuracy: 0.7763
Epoch 7/20
101/101 [==============================] - 56s 554ms/step - loss: 0.4727 - accuracy: 0.7764 - val_loss: 0.4534 - val_ac

In [39]:
# print(df_prob_train['CNN-CV-0'])
# # for ext in ['CV', 'TFIDF', 'W2V', 'GLOVE', 'FAST']:
# #   for i in range(0,3):
# #     df_prob_train = df_prob_train.drop(columns=['CNN-{}-{}'.format(ext, i)])
# #     df_prob_val = df_prob_val.drop(columns=['CNN-{}-{}'.format(ext, i)])
# #     df_prob_test = df_prob_test.drop(columns=['CNN-{}-{}'.format(ext, i)])

In [40]:
print(df_prob_train.columns)

Index(['class', 'SVM-CV-0', 'SVM-CV-1', 'SVM-CV-2', 'SVM-TFIDF-0',
       'SVM-TFIDF-1', 'SVM-TFIDF-2', 'SVM-W2V-0', 'SVM-W2V-1', 'SVM-W2V-2',
       ...
       'KNN-TFIDF-2', 'KNN-W2V-0', 'KNN-W2V-1', 'KNN-W2V-2', 'KNN-GLOVE-0',
       'KNN-GLOVE-1', 'KNN-GLOVE-2', 'KNN-FAST-0', 'KNN-FAST-1', 'KNN-FAST-2'],
      dtype='object', length=106)


In [41]:
# df_pred_test = df_pred_test.drop(['CNN-CV'], axis=1)
# print(df_pred_test.columns)
# CV
cols = ["CNN-CV-0", "CNN-CV-1", "CNN-CV-2"]
df_train_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_train), axis=1), columns=["CNN-CV"])
df_val_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_val), axis=1), columns=["CNN-CV"])
df_test_ = pd.DataFrame(np.argmax(cnn_cv.predict(data_test), axis=1), columns=["CNN-CV"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# print(df_pred_test.columns)

# # Probabilidades
  
df_train_ = pd.DataFrame(cnn_cv.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_cv.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_cv.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


cols = ["CNN-TF-0", "CNN-TF-1", "CNN-TF-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_train), axis=1), columns=["CNN-TFIDF"])
df_val_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_val), axis=1), columns=["CNN-TFIDF"])
df_test_ = pd.DataFrame(np.argmax(cnn_tfidf.predict(data_test), axis=1), columns=["CNN-TFIDF"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_tfidf.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_tfidf.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_tfidf.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

cols = ["CNN-W2V-0", "CNN-W2V-1", "CNN-W2V-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_train), axis=1), columns=["CNN-W2V"])
df_val_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_val), axis=1), columns=["CNN-W2V"])
df_test_ = pd.DataFrame(np.argmax(cnn_w2v.predict(data_test), axis=1), columns=["CNN-W2V"])
  
df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_w2v.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_w2v.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_w2v.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


cols = ["CNN-GLOVE-0", "CNN-GLOVE-1", "CNN-GLOVE-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_train), axis=1), columns=["CNN-GLOVE"])
df_val_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_val), axis=1), columns=["CNN-GLOVE"])
df_test_ = pd.DataFrame(np.argmax(cnn_glove.predict(data_test), axis=1), columns=["CNN-GLOVE"])

df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_glove.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_glove.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_glove.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)

cols = ["CNN-FAST-0", "CNN-FAST-1", "CNN-FAST-2"]

df_train_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_train), axis=1), columns=["CNN-FAST"])
df_val_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_val), axis=1), columns=["CNN-FAST"])
df_test_ = pd.DataFrame(np.argmax(cnn_fast.predict(data_test), axis=1), columns=["CNN-FAST"])

df_pred_train = pd.concat([df_pred_train, df_train_], axis=1, sort=False)
df_pred_val = pd.concat([df_pred_val, df_val_], axis=1, sort=False)
df_pred_test = pd.concat([df_pred_test, df_test_], axis=1, sort=False)

# Probabilidades
  
df_train_ = pd.DataFrame(cnn_fast.predict(data_train), columns=cols)
df_val_ = pd.DataFrame(cnn_fast.predict(data_val), columns=cols)
df_test_ = pd.DataFrame(cnn_fast.predict(data_test), columns=cols)

df_prob_train = pd.concat([df_prob_train, df_train_], axis=1, sort=False)
df_prob_val = pd.concat([df_prob_val, df_val_], axis=1, sort=False)
df_prob_test = pd.concat([df_prob_test, df_test_], axis=1, sort=False)


In [42]:
df_prob_train['class']

0        2
1        1
2        1
3        1
4        1
        ..
20182    1
20183    0
20184    0
20185    2
20186    1
Name: class, Length: 20187, dtype: int64

In [43]:
df_pred_train.to_csv("pred_train_td_zw.csv")
df_pred_val.to_csv("pred_val_td_zw.csv")
df_pred_test.to_csv("pred_test_td_zw.csv")

In [44]:
df_prob_train.to_csv("prob_train_td_zw.csv")
df_prob_val.to_csv("prob_val_td_zw.csv")
df_prob_test.to_csv("prob_test_td_zw.csv")

In [45]:
df_pred_val.shape

(4972, 41)